In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import holidays # Para analise de feriados
import warnings

# Configuracoes basicas
warnings.filterwarnings('ignore') # Suprime avisos comuns do pandas/seaborn
pd.set_option('display.max_columns', 50) # Mostrar mais colunas
sns.set_style('whitegrid') # Estilo dos graficos seaborn
plt.rcParams['figure.figsize'] = (12, 6) # Tamanho padrao das figuras matplotlib

print("Bibliotecas importadas e configuracoes basicas aplicadas.")

Bibliotecas importadas e configuracoes basicas aplicadas.


In [3]:
print("Carregando arquivos CSV...")

# Caminhos para os arquivos (ajuste se necessario)
path_product = './data/product.csv'
path_address = './data/rideaddress_v1.csv' # Usaremos apenas para referencia inicial, se necessario
path_ride_v2 = './data/ride_v2.csv'
path_estimative = './data/rideestimative_v3.csv'
# <<< IMPORTANTE: Confirme o nome exato do arquivo gerado pelo script ORS >>>
path_ors_results = './data/ors_local_coords_only_results.csv' # Ou o nome que voce salvou

try:
    # Carrega com separador e tratamento de encoding
    df_prod = pd.read_csv(path_product, sep=';', encoding='utf-8')
except UnicodeDecodeError:
    print("WARN: Falha UTF-8 em product.csv, tentando latin-1...")
    df_prod = pd.read_csv(path_product, sep=';', encoding='latin-1')
except FileNotFoundError:
    print(f"ERRO: Arquivo nao encontrado: {path_product}")
    df_prod = pd.DataFrame() # Cria df vazio para evitar erros posteriores

# try:
#     df_addr = pd.read_csv(path_address, sep=';', encoding='utf-8', low_memory=False)
# except UnicodeDecodeError:
#     print("WARN: Falha UTF-8 em rideaddress_v1.csv, tentando latin-1...")
#     df_addr = pd.read_csv(path_address, sep=';', encoding='latin-1', low_memory=False)
# except FileNotFoundError:
#     print(f"ERRO: Arquivo nao encontrado: {path_address}")
#     df_addr = pd.DataFrame()

try:
    df_rides = pd.read_csv(path_ride_v2, sep=';', encoding='utf-8', low_memory=False)
except UnicodeDecodeError:
    print("WARN: Falha UTF-8 em ride_v2.csv, tentando latin-1...")
    df_rides = pd.read_csv(path_ride_v2, sep=';', encoding='latin-1', low_memory=False)
except FileNotFoundError:
    print(f"ERRO: Arquivo nao encontrado: {path_ride_v2}")
    df_rides = pd.DataFrame()

try:
    df_estim = pd.read_csv(path_estimative, sep=';', encoding='utf-8', low_memory=False)
except UnicodeDecodeError:
    print("WARN: Falha UTF-8 em rideestimative_v3.csv, tentando latin-1...")
    df_estim = pd.read_csv(path_estimative, sep=';', encoding='latin-1', low_memory=False)
except FileNotFoundError:
    print(f"ERRO: Arquivo nao encontrado: {path_estimative}")
    df_estim = pd.DataFrame()

try:
    # Assumindo que o script ORS salvou com ; como separador
    df_ors = pd.read_csv(path_ors_results, sep=';', encoding='utf-8')
except UnicodeDecodeError:
    print(f"WARN: Falha UTF-8 em {path_ors_results}, tentando latin-1...")
    df_ors = pd.read_csv(path_ors_results, sep=';', encoding='latin-1')
except FileNotFoundError:
    print(f"ERRO: Arquivo nao encontrado: {path_ors_results}. Execute o script 'calcular_rotas_ors.py' primeiro.")
    df_ors = pd.DataFrame()
except Exception as e:
     print(f"ERRO ao carregar {path_ors_results}. Verifique o separador (sep=';') ou o arquivo. Erro: {e}")
     df_ors = pd.DataFrame()


# Verifica se os dataframes principais foram carregados
if df_rides.empty or df_estim.empty or df_ors.empty:
    print("\n!!! ERRO CRITICO: Nao foi possivel carregar um ou mais dos arquivos CSV principais (rides, estim, ors_results). Verifique os caminhos e erros acima. !!!")
else:
    print("\nCSVs carregados. Primeiras linhas:")
    print("\n--- df_rides (ride_v2) ---")
    print(df_rides.head(2))
    print(f"\nShape: {df_rides.shape}")
    print("\n--- df_estim (rideestimative_v3) ---")
    print(df_estim.head(2))
    print(f"\nShape: {df_estim.shape}")
    print("\n--- df_ors (resultados ORS local) ---")
    print(df_ors.head(2))
    print(f"\nShape: {df_ors.shape}")
    if not df_prod.empty:
        print("\n--- df_prod (product) ---")
        print(df_prod.head(2))
        print(f"\nShape: {df_prod.shape}")

Carregando arquivos CSV...

CSVs carregados. Primeiras linhas:

--- df_rides (ride_v2) ---
    RideID                                UserID                     Schedule  \
0  1685755  e15b8cc3-5a67-4630-b89f-ee69f302b582  2025-02-10 14:31:10.8858446   
1  1685754  5c3fb011-0aea-429a-8305-b88953b77df1  2025-02-10 14:26:35.3411403   

                        Create  RideStatusID  CompanyID  ProviderID  \
0  2025-02-10 14:31:10.9084221             1          2         NaN   
1  2025-02-10 14:26:35.4169873             2        230         5.0   

  RideProviderID  price                      Updated  CategoryID  TotalUsers  \
0            NaN   0.00  2025-02-10 14:31:10.9084233         NaN           1   
1            NaN  30.45  2025-02-10 14:28:02.4656963         NaN           1   

   Car  RideDriverLocationID  ScheduledRide  
0  NaN                   NaN              0  
1  NaN                   NaN              0  

Shape: (500000, 15)

--- df_estim (rideestimative_v3) ---
   RideEstima

In [4]:
# Bloco 3: Inspecao inicial, selecao e renomeacao de colunas

print("\n--- Informacoes df_rides ---")
df_rides.info(memory_usage='deep')

print("\n--- Informacoes df_estim ---")
df_estim.info(memory_usage='deep')

print("\n--- Informacoes df_ors ---")
df_ors.info(memory_usage='deep')

# Selecionar colunas relevantes e renomear para padrao
# De df_rides: RideID, Create (data/hora), ProviderID
if not df_rides.empty:
    rides_cols = {
        "RideID": "ride_id",
        "Create": "datetime_str", # Usaremos 'Create' como timestamp principal
        "ProviderID": "provider_id"
        # "Schedule": "schedule_str" # Pode adicionar se quiser comparar
    }
    # Filtra apenas colunas existentes no dataframe lido
    cols_to_select_rides = [col for col in rides_cols.keys() if col in df_rides.columns]
    if len(cols_to_select_rides) < len(rides_cols):
        print(f"WARN: Colunas ausentes em df_rides: {set(rides_cols.keys()) - set(cols_to_select_rides)}")

    df_rides_clean = df_rides[cols_to_select_rides].copy()
    df_rides_clean.rename(columns=rides_cols, inplace=True)
    print("\nColunas selecionadas/renomeadas de df_rides:")
    print(df_rides_clean.head(2))
else:
    df_rides_clean = pd.DataFrame(columns=['ride_id', 'datetime_str', 'provider_id']) # Cria vazio se falhou ao carregar

# De df_estim: RideID, ProductID, Price (TARGET)
if not df_estim.empty:
    estim_cols = {
        "RideID": "ride_id",
        "ProductID": "product_id",
        "Price": "price" # Nossa variavel alvo
    }
    cols_to_select_estim = [col for col in estim_cols.keys() if col in df_estim.columns]
    if len(cols_to_select_estim) < len(estim_cols):
         print(f"WARN: Colunas ausentes em df_estim: {set(estim_cols.keys()) - set(cols_to_select_estim)}")

    df_estim_clean = df_estim[cols_to_select_estim].copy()
    df_estim_clean.rename(columns=estim_cols, inplace=True)
    print("\nColunas selecionadas/renomeadas de df_estim:")
    print(df_estim_clean.head(2))
else:
    df_estim_clean = pd.DataFrame(columns=['ride_id', 'product_id', 'price'])

# De df_ors: ride_id, distance_m, duration_s (ja vem com nomes bons)
if not df_ors.empty:
    # Garante que ride_id seja numerico (geralmente int) para merge
    # Tenta converter, mas lida com possiveis erros se ja for numerico
    try:
         if not pd.api.types.is_numeric_dtype(df_ors['ride_id']):
              df_ors['ride_id'] = pd.to_numeric(df_ors['ride_id'], errors='coerce')
              df_ors.dropna(subset=['ride_id'], inplace=True) # Remove se nao pode converter
              df_ors['ride_id'] = df_ors['ride_id'].astype(int)
    except KeyError:
         print("ERRO: Coluna 'ride_id' nao encontrada em df_ors.")
    except Exception as e:
         print(f"ERRO ao processar 'ride_id' em df_ors: {e}")

    # Renomeia colunas explicitamente para garantir
    ors_cols = {
        "ride_id": "ride_id", # Mantem
        "distance_m": "distance_m", # Mantem (ou o nome que seu script gerou)
        "duration_s": "duration_s" # Mantem (ou o nome que seu script gerou)
        # Inclua 'ors_error_detail' se quiser analisar erros
    }
    cols_to_select_ors = [col for col in ors_cols.keys() if col in df_ors.columns]
    if len(cols_to_select_ors) < len(ors_cols):
        print(f"WARN: Colunas ausentes em df_ors: {set(ors_cols.keys()) - set(cols_to_select_ors)}")

    df_ors_clean = df_ors[cols_to_select_ors].copy()
    df_ors_clean.rename(columns=ors_cols, inplace=True)
    print("\nColunas selecionadas/renomeadas de df_ors:")
    print(df_ors_clean.head(2))
else:
     df_ors_clean = pd.DataFrame(columns=['ride_id', 'distance_m', 'duration_s'])

# De df_prod: ProductID, ProviderID, Description
if not df_prod.empty:
    prod_cols = {
        "ProductID": "product_id",
        "ProviderID": "provider_id_prod", # Renomeia para evitar conflito
        "Description": "product_description"
    }
    cols_to_select_prod = [col for col in prod_cols.keys() if col in df_prod.columns]
    if len(cols_to_select_prod) < len(prod_cols):
         print(f"WARN: Colunas ausentes em df_prod: {set(prod_cols.keys()) - set(cols_to_select_prod)}")

    df_prod_clean = df_prod[cols_to_select_prod].copy()
    df_prod_clean.rename(columns=prod_cols, inplace=True)
    print("\nColunas selecionadas/renomeadas de df_prod:")
    print(df_prod_clean.head(2))
else:
     df_prod_clean = pd.DataFrame(columns=['product_id', 'provider_id_prod', 'product_description'])


--- Informacoes df_rides ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   RideID                500000 non-null  int64  
 1   UserID                500000 non-null  object 
 2   Schedule              500000 non-null  object 
 3   Create                500000 non-null  object 
 4   RideStatusID          500000 non-null  int64  
 5   CompanyID             500000 non-null  int64  
 6   ProviderID            228157 non-null  float64
 7   RideProviderID        21440 non-null   object 
 8   price                 500000 non-null  float64
 9   Updated               500000 non-null  object 
 10  CategoryID            24714 non-null   float64
 11  TotalUsers            500000 non-null  int64  
 12  Car                   14944 non-null   object 
 13  RideDriverLocationID  14864 non-null   float64
 14  ScheduledRide         

In [5]:
# Bloco 4: Merging dos DataFrames

print("\n4. Realizando merges...")

# DataFrame base: df_estim_clean (cada linha e uma estimativa de preco)
if df_estim_clean.empty:
    print("ERRO: df_estim_clean esta vazio. Nao e possivel continuar.")
    df_merged = pd.DataFrame()
else:
    print(f"   df_estim_clean inicial: {df_estim_clean.shape}")

    # 1. Merge com df_rides_clean (para obter datetime_str e provider_id)
    if not df_rides_clean.empty:
        df_merged = pd.merge(df_estim_clean, df_rides_clean, on='ride_id', how='inner')
        print(f"   Apos merge com df_rides_clean: {df_merged.shape}")
        # Verifica quantos ride_id unicos sobraram
        print(f"      → {df_merged['ride_id'].nunique()} ride_ids unicos.")
    else:
        print("   WARN: df_rides_clean vazio, pulando merge.")
        df_merged = df_estim_clean # Continua so com estimativas
        # Adiciona colunas faltantes com NaN para consistencia
        if 'datetime_str' not in df_merged.columns: df_merged['datetime_str'] = np.nan
        if 'provider_id' not in df_merged.columns: df_merged['provider_id'] = np.nan


    # 2. Merge com df_ors_clean (para obter distance_m e duration_s)
    if not df_ors_clean.empty and not df_merged.empty:
        # Antes do merge, verifica tipo de ride_id
        if not pd.api.types.is_numeric_dtype(df_merged['ride_id']):
             print("   WARN: Convertendo ride_id em df_merged para numerico antes do merge com ORS.")
             df_merged['ride_id'] = pd.to_numeric(df_merged['ride_id'], errors='coerce')
             df_merged.dropna(subset=['ride_id'], inplace=True)
             df_merged['ride_id'] = df_merged['ride_id'].astype(int)

        # Garante que ride_id em df_ors_clean tambem e int
        if 'ride_id' in df_ors_clean.columns and not pd.api.types.is_integer_dtype(df_ors_clean['ride_id']):
             df_ors_clean['ride_id'] = df_ors_clean['ride_id'].astype(int)

        # Remove duplicatas em df_ors_clean antes do merge (seguranca)
        df_ors_clean_unique = df_ors_clean.drop_duplicates(subset=['ride_id'], keep='first')

        df_merged = pd.merge(df_merged, df_ors_clean_unique, on='ride_id', how='inner')
        print(f"   Apos merge com df_ors_clean: {df_merged.shape}")
        print(f"      → {df_merged['ride_id'].nunique()} ride_ids unicos.")
        # Verifica NaNs introduzidos
        print(f"      → NaNs em distance_m: {df_merged['distance_m'].isna().sum()}")
        print(f"      → NaNs em duration_s: {df_merged['duration_s'].isna().sum()}")
    else:
        print("   WARN: df_ors_clean ou df_merged vazio, pulando merge ORS.")
        # Adiciona colunas faltantes com NaN
        if 'distance_m' not in df_merged.columns: df_merged['distance_m'] = np.nan
        if 'duration_s' not in df_merged.columns: df_merged['duration_s'] = np.nan

    # 3. Merge com df_prod_clean (opcional, para obter nome do produto/provedor)
    if not df_prod_clean.empty and not df_merged.empty:
        # Limpeza de product_id (pode ser string ou numero)
        # Tenta converter ambos para string para o merge
        try:
             df_merged['product_id_str'] = df_merged['product_id'].astype(str).str.strip()
             df_prod_clean['product_id_str'] = df_prod_clean['product_id'].astype(str).str.strip()

             df_merged = pd.merge(df_merged,
                                   df_prod_clean[['product_id_str', 'product_description', 'provider_id_prod']],
                                   on='product_id_str',
                                   how='left') # Usa left join para manter todas as estimativas
             print(f"   Apos merge com df_prod_clean: {df_merged.shape}")
             df_merged.drop(columns=['product_id_str'], inplace=True) # Remove coluna temporaria

             # Verifica consistencia de provider_id (se ambos existem)
             if 'provider_id' in df_merged.columns and 'provider_id_prod' in df_merged.columns:
                 provider_mismatch = df_merged[df_merged['provider_id'].notna() &
                                               df_merged['provider_id_prod'].notna() &
                                               (df_merged['provider_id'] != df_merged['provider_id_prod'])].shape[0]
                 if provider_mismatch > 0:
                     print(f"   WARN: {provider_mismatch} linhas com provider_id inconsistente entre ride_v2 e product.")
                     # Pode ser necessario decidir qual provider_id usar ou investigar
                 # Opcional: Usar o provider_id do product se o de ride_v2 for nulo
                 # df_merged['provider_id'] = df_merged['provider_id'].fillna(df_merged['provider_id_prod'])

        except Exception as e:
             print(f"   WARN: Erro ao fazer merge com df_prod_clean: {e}. Pulando este merge.")
    else:
        print("   INFO: df_prod_clean vazio ou df_merged vazio, pulando merge com produtos.")


# Exibe resultado final do merge
if not df_merged.empty:
    print("\nDataFrame final apos merges:")
    print(df_merged.head())
    print(f"\nShape final: {df_merged.shape}")
    print("\nInformacoes do DataFrame merged:")
    df_merged.info(memory_usage='deep')
    print("\nValores nulos por coluna:")
    print(df_merged.isnull().sum())
else:
    print("ERRO: Falha em algum ponto dos merges. DataFrame resultante esta vazio.")

# Limpa dataframes intermediarios para liberar memoria (opcional)
# del df_rides_clean, df_estim_clean, df_ors_clean, df_prod_clean
# del df_rides, df_estim, df_ors, df_prod
# import gc
# gc.collect()


4. Realizando merges...
   df_estim_clean inicial: (2000000, 3)
   Apos merge com df_rides_clean: (2000000, 5)
      → 239270 ride_ids unicos.
   Apos merge com df_ors_clean: (1999989, 7)
      → 239268 ride_ids unicos.
      → NaNs em distance_m: 520
      → NaNs em duration_s: 520
   Apos merge com df_prod_clean: (1999989, 10)
   WARN: 11683 linhas com provider_id inconsistente entre ride_v2 e product.

DataFrame final apos merges:
   ride_id product_id   price                 datetime_str  provider_id  \
0  1183200      Flash   89.00  2021-08-17 10:09:45.6425825          NaN   
1  1183200      UberX   89.00  2021-08-17 10:09:45.6425825          NaN   
2  1183200    Comfort  116.50  2021-08-17 10:09:45.6425825          NaN   
3  1183200    poupa99  170.21  2021-08-17 10:09:45.6425825          NaN   
4  1183200      pop99  170.21  2021-08-17 10:09:45.6425825          NaN   

   distance_m  duration_s product_description  provider_id_prod  
0    41121.25     2978.98               Flas

In [7]:
df_merged.head(30) # Exibe as primeiras linhas do DataFrame final

,ride_id,product_id,price,datetime_str,provider_id,distance_m,duration_s,product_description,provider_id_prod
0,1183200,Flash,89.00,2021-08-17 10:09:45.6425825,NaN,41121.25,2978.98,Flash,2
1,1183200,UberX,89.00,2021-08-17 10:09:45.6425825,NaN,41121.25,2978.98,UberX,2
2,1183200,Comfort,116.50,2021-08-17 10:09:45.6425825,NaN,41121.25,2978.98,Comfort,2
3,1183200,poupa99,170.21,2021-08-17 10:09:45.6425825,NaN,41121.25,2978.98,99POUPA,3
4,1183200,pop99,170.21,2021-08-17 10:09:45.6425825,NaN,41121.25,2978.98,99POP,3
5,1183200,turbo-taxi,151.05,2021-08-17 10:09:45.6425825,NaN,41121.25,2978.98,99TAXI,3
6,1183200,regular-taxi,151.05,2021-08-17 10:09:45.6425825,NaN,41121.25,2978.98,Táxi Comum,3
7,1183201,Flash,31.50,2021-08-17 10:09:52.0369351,NaN,17864.57,1413.11,Flash,2
8,1183201,Comfort,33.50,2021-08-17 10:09:52.0369351,NaN,17864.57,1413.11,Comfort,2
9,1183201,poupa99,26.91,2021-08-17 10:09:52.0369351,NaN,17864.57,1413.11,99POUPA,3
